In [2]:
import os
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
import faiss
import pickle
import sqlite3

# Chemins
DB_PATH = "arxiv_relational.db"        # Base de données SQLite locale
FAISS_INDEX_PATH = "faiss_index.bin"   # Fichier d'index FAISS
ARTICLE_IDS_PATH = "article_ids.pkl"   # Mapping article_id <-> position FAISS

# 1. Chargement des articles depuis SQLite
print("🔹 Chargement des articles depuis SQLite...")
conn = sqlite3.connect(DB_PATH)
df = pd.read_sql_query("SELECT article_id, title, abstract FROM articles", conn)
conn.close()
print(f"🔹 Articles chargés : {len(df)}")

# 2. Nettoyage simple des textes
def preprocess_text(text):
    if pd.isna(text):
        return ""
    return text.replace('\n', ' ').strip()

texts = ((df['title'].fillna('') + ". " + df['abstract'].fillna('')).map(preprocess_text)).tolist()

# 3. Vectorisation avec Sentence Transformers
print("🔹 Chargement du modèle SentenceTransformer...")
model = SentenceTransformer('all-MiniLM-L6-v2')

print("🔹 Vectorisation des textes...")
embeddings = model.encode(texts, batch_size=64, show_progress_bar=True, normalize_embeddings=True)

# 4. Création de l’index FAISS
print("🔹 Création de l'index FAISS (cosine similarity)...")
dimension = embeddings.shape[1]
index = faiss.IndexFlatIP(dimension)
index.add(embeddings)

# 5. Sauvegarde de l'index et des IDs
os.makedirs("data", exist_ok=True)
faiss.write_index(index, FAISS_INDEX_PATH)
with open(ARTICLE_IDS_PATH, 'wb') as f:
    pickle.dump(df['article_id'].tolist(), f)

print("✅ Index FAISS sauvegardé :", FAISS_INDEX_PATH)
print("✅ Liste des article_id sauvegardée :", ARTICLE_IDS_PATH)



🔹 Chargement des articles depuis SQLite...
🔹 Articles chargés : 3000
🔹 Chargement du modèle SentenceTransformer...
🔹 Vectorisation des textes...


Batches:   0%|          | 0/47 [00:00<?, ?it/s]

🔹 Création de l'index FAISS (cosine similarity)...
✅ Index FAISS sauvegardé : faiss_index.bin
✅ Liste des article_id sauvegardée : article_ids.pkl
